In [1]:
# Some of the methods for this problem were reused from homework 5

import pickle
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
import numpy as np
from keras import optimizers

Using TensorFlow backend.


In [2]:
letter_data = pickle.load(open("data/letter_data.p", "rb"))

In [3]:
# Dictionaries to convert between letter and index
letter_int = {}
int_letter = {}
i = 0
for poem in letter_data:
    for letter in poem:
        if letter not in letter_int:
            letter_int[letter] = i
            int_letter[i] = letter
            i += 1

In [4]:
# Returns one-hot-encoded feature representation of the specified word given
# a dictionary mapping words to their one-hot-encoded index.
def get_word_repr(letter_to_int, word):
    unique_words = letter_to_int.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[letter_to_int[word]] = 1
    return feature_representation

In [5]:
def generate_traindata(word_list, word_to_index, window_size=40, skip = 3):
    """
    Generates training data for Skipgram model (sort of).

    Arguments:
        word_list:     Sequential list of letters (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window.
        
        skip:          Skip every skip characters 

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training
                              points (one-hot-encoded vectors representing individual words) and
                              their corresponding labels (also one-hot-encoded vectors representing words).
    """
    trainX = []
    trainY = []
    for i in range(window_size, len(word_list), skip):
        curr_word = word_list[i]
        curr_X = []
        for j in range(-window_size, 0):
            if j != 0 and i + j >= 0 and i + j < len(word_list):
                adjacent_word = word_list[i + j]
                curr_X.append(get_word_repr(word_to_index, adjacent_word))
        trainX.append(curr_X)
        trainY.append(get_word_repr(word_to_index, curr_word))
        
    return (np.array(trainX), np.array(trainY))

In [6]:
# Create the training set
unit = True
train_x = -1
train_y = -1
for poem in letter_data:
    poem_train_x, poem_train_y = generate_traindata(poem, letter_int)
    if unit:
        train_x = poem_train_x
        train_y = poem_train_y
        unit = False
    else:
        train_x = np.concatenate((train_x, poem_train_x))
        train_y = np.concatenate((train_y, poem_train_y))

In [7]:
# Generate the neural network
model = Sequential()
model.add(LSTM(150, input_shape = (len(train_x[0]), len(train_x[0][0]))))
model.add(Dense(len(train_y[0])))
model.add(Activation('softmax'))
model.summary()

model.compile(loss = "categorical_crossentropy", optimizer = "rmsprop", metrics = ['accuracy'])
model.fit(train_x, train_y, epochs = 250, batch_size = 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 150)               113400    
_________________________________________________________________
dense_1 (Dense)              (None, 38)                5738      
_________________________________________________________________
activation_1 (Activation)    (None, 38)                0         
Total params: 119,138
Trainable params: 119,138
Non-trainable params: 0
_________________________________________________________________
Epoch 1/250
29233/29233 [==============================] - 38s 1ms/step - loss: 2.9100 - acc: 0.1875
Epoch 2/250
29233/29233 [==============================] - 26s 895us/step - loss: 2.4828 - acc: 0.2994
Epoch 3/250
29233/29233 [==============================] - 23s 777us/step - loss: 2.2926 - acc: 0.3363
Epoch 4/250
29233/29233 [==============================] - 35s 1ms/step - loss: 2.1934 - acc:

29233/29233 [==============================] - 20s 675us/step - loss: 0.1951 - acc: 0.9546
Epoch 74/250
29233/29233 [==============================] - 20s 676us/step - loss: 0.1894 - acc: 0.9574
Epoch 75/250
29233/29233 [==============================] - 20s 675us/step - loss: 0.1857 - acc: 0.9580
Epoch 76/250
29233/29233 [==============================] - 20s 674us/step - loss: 0.1839 - acc: 0.9589
Epoch 77/250
29233/29233 [==============================] - 20s 674us/step - loss: 0.1773 - acc: 0.9593
Epoch 78/250
29233/29233 [==============================] - 20s 674us/step - loss: 0.1757 - acc: 0.9587
Epoch 79/250
29233/29233 [==============================] - 20s 681us/step - loss: 0.1696 - acc: 0.9617
Epoch 80/250
29233/29233 [==============================] - 20s 675us/step - loss: 0.1690 - acc: 0.9615
Epoch 81/250
29233/29233 [==============================] - 20s 675us/step - loss: 0.1654 - acc: 0.9618
Epoch 82/250
29233/29233 [==============================] - 20s 673us/step - 

29233/29233 [==============================] - 37s 1ms/step - loss: 0.0859 - acc: 0.9778
Epoch 152/250
29233/29233 [==============================] - 34s 1ms/step - loss: 0.0888 - acc: 0.9774
Epoch 153/250
29233/29233 [==============================] - 21s 721us/step - loss: 0.0921 - acc: 0.9761
Epoch 154/250
29233/29233 [==============================] - 34s 1ms/step - loss: 0.0870 - acc: 0.9781
Epoch 155/250
29233/29233 [==============================] - 37s 1ms/step - loss: 0.0859 - acc: 0.9776
Epoch 156/250
29233/29233 [==============================] - 37s 1ms/step - loss: 0.0858 - acc: 0.9775
Epoch 157/250
29233/29233 [==============================] - 37s 1ms/step - loss: 0.0877 - acc: 0.9770
Epoch 158/250
29233/29233 [==============================] - 38s 1ms/step - loss: 0.0848 - acc: 0.9787
Epoch 159/250
29233/29233 [==============================] - 37s 1ms/step - loss: 0.0836 - acc: 0.9782
Epoch 160/250
29233/29233 [==============================] - 37s 1ms/step - loss: 0.0

29233/29233 [==============================] - 22s 761us/step - loss: 0.0628 - acc: 0.9830
Epoch 231/250
29233/29233 [==============================] - 24s 834us/step - loss: 0.0668 - acc: 0.9817
Epoch 232/250
29233/29233 [==============================] - 21s 732us/step - loss: 0.0654 - acc: 0.9812
Epoch 233/250
29233/29233 [==============================] - 22s 737us/step - loss: 0.0632 - acc: 0.9836
Epoch 234/250
29233/29233 [==============================] - 22s 765us/step - loss: 0.0684 - acc: 0.9816
Epoch 235/250
29233/29233 [==============================] - 22s 739us/step - loss: 0.0679 - acc: 0.9815
Epoch 236/250
29233/29233 [==============================] - 21s 723us/step - loss: 0.0617 - acc: 0.9831
Epoch 237/250
29233/29233 [==============================] - 22s 759us/step - loss: 0.0689 - acc: 0.9809
Epoch 238/250
29233/29233 [==============================] - 24s 834us/step - loss: 0.0638 - acc: 0.9822
Epoch 239/250
29233/29233 [==============================] - 21s 707u

In [ ]:
model.fit(train_x, train_y, epochs = 100, batch_size = 64)

Epoch 1/100
29233/29233 [==============================] - 24s 830us/step - loss: 0.0960 - acc: 0.9702
Epoch 2/100
29233/29233 [==============================] - 25s 839us/step - loss: 0.0927 - acc: 0.9717
Epoch 3/100
29233/29233 [==============================] - 25s 857us/step - loss: 0.0951 - acc: 0.9707
Epoch 4/100
29233/29233 [==============================] - 25s 852us/step - loss: 0.0926 - acc: 0.9716
Epoch 5/100
29233/29233 [==============================] - 25s 862us/step - loss: 0.0930 - acc: 0.9719
Epoch 6/100
29233/29233 [==============================] - 25s 872us/step - loss: 0.0942 - acc: 0.9717
Epoch 7/100
29233/29233 [==============================] - 25s 848us/step - loss: 0.0903 - acc: 0.9736
Epoch 8/100
29233/29233 [==============================] - 24s 832us/step - loss: 0.0927 - acc: 0.9711
Epoch 9/100
29233/29233 [==============================] - 25s 853us/step - loss: 0.0977 - acc: 0.9703
Epoch 10/100
29233/29233 [==============================] - 25s 866us/ste

In [ ]:
# Convert a line to one hot form
def convert_line_to_one_hot(line, letter_int):
    output = []
    for letter in line:
        output.append(get_word_repr(letter_int, letter))
    return np.array(output)

In [ ]:
# Convert from one hot to letter
def vect_to_letter(vect, int_letter):
    max_index = 0
    max_value = 0
    for curr_index, curr_value in enumerate(vect):
        if max_value < curr_value:
            max_index = curr_index
            max_value = curr_value
    return int_letter[max_index]

In [ ]:
# Given a RNN generates a shakespeare sonnet
def generate_poem(RNN, letter_int, int_letter):
    # Initial line to seed 
    first_line = "shall i compare thee to a summer\'s day?\n"
    
    curr_line = convert_line_to_one_hot(first_line, letter_int)
    curr_poem = first_line
    line_num = 2
    while line_num < 15:
        # Predict the next character
        next_char = RNN.predict(np.array([curr_line]))[0]
        # Update curr_line and curr_poem
        curr_poem += vect_to_letter(next_char, int_letter)
        if vect_to_letter(next_char, int_letter) == "\n":
            line_num += 1
        curr_line = convert_line_to_one_hot(curr_poem[-40:], letter_int)
    
    return curr_poem

In [ ]:
poem = generate_poem(model, letter_int, int_letter)
print(poem)